In [1]:
import os
import tensorflow as tf
import numpy as np
import yaml
from tensorflow import keras

import matplotlib.pyplot as plt
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

tf.enable_eager_execution()

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
OUTPUT_DIR_PATH = '/work/nas/emotion/02_TFRecords/smile_all_v2_224_190422'
TRAIN_OUTPUT_DIR_PATH = OUTPUT_DIR_PATH + "/train"
VAL_OUTPUT_DIR_PATH = OUTPUT_DIR_PATH + "/val"

LABEL_PATH = '/work/nas/emotion/02_TFRecords/smile_all_v2_224_190422/label.txt' # Could be file or dir that has folders named as labels

DEBUG_IMAGES = False

TRAIN_BATCH_SIZE = 32

INPUT_SIZE = 224
INPUT_CHANNELS = 3
NUM_CLASSES = 3
BATCH_EPOCHS = 1000
VAL_BATCH_SIZE = 64

CKPT_DIR = '/work/nas/emotion/03_CheckPoint/smile_v2_224_mobilenet_190522/ckpt'
LOAD_CKPT = False

In [3]:
class MobileNetV1 :
    def __init__(self, learning_rate=0.01, lr_min=0.0001, lr_decay=0.94, lr_decay_epoch=5) :
        self.learning_rate = learning_rate;
        self.lr_min = lr_min
        self.lr_decay = lr_decay
        self.lr_decay_epoch = lr_decay_epoch

    def separable_conv2D(self, x, channels, strides=1) :
        x = keras.layers.DepthwiseConv2D(3, strides, padding='same')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation('relu')(x)
        x = keras.layers.Conv2D(channels, 1, 1, padding='same')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation('relu')(x)
        return x
    
    def create_model(self) :
        inputs = tf.keras.layers.Input(shape=(224, 224, 3), name='x')
        x = keras.layers.Conv2D(32, 3, 2, padding='same')(inputs) # 112
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation('relu')(x)
        x = self.separable_conv2D(x, 64)
        x = self.separable_conv2D(x, 128, 2) # 56
        x = self.separable_conv2D(x, 128)
        x = self.separable_conv2D(x, 256, 2) # 28
        x = self.separable_conv2D(x, 256)
        x = self.separable_conv2D(x, 512, 2) # 14
        for idx in range(5) :
            x = self.separable_conv2D(x, 512)
        x = self.separable_conv2D(x, 1024, 2) # 7
        x = self.separable_conv2D(x, 1024)
        x = keras.layers.AveragePooling2D(7)(x)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(1000, activation=tf.keras.activations.relu)(x)
        x = keras.layers.Dropout(0.2)(x)
        x = keras.layers.Dense(NUM_CLASSES, activation=tf.keras.activations.softmax, name='final')(x)
           
        model = keras.Model(inputs=[inputs], outputs=[x])
#         model = keras.utils.multi_gpu_model(model, gpus=2)
        model.compile(optimizer=tf.train.AdamOptimizer(self.learning_rate), 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
        return model

In [4]:
def create_dataset_from_tfrecord(data_dir_path, batch_size) :
    tfrecord_file_paths = tf.gfile.Glob('%s/*.tfrecord' % (data_dir_path))
    dataset = tf.data.TFRecordDataset(tfrecord_file_paths)
    dataset = dataset.map(parse)
    dataset = dataset.map(preprocess_dataset)
#     dataset = dataset.repeat() # This dataset will go on forever   
    dataset = dataset.shuffle(1000) # Set the number of datapoints you want to load and shuffle 
    dataset = dataset.batch(batch_size) # Set the batchsize
    
    return dataset

def parse(data):
    features = {"image/encoded": tf.FixedLenFeature((), tf.string, default_value=""),
                "image/class/label": tf.FixedLenFeature((), tf.int64, default_value=0),
                "image/height": tf.FixedLenFeature((), tf.int64, default_value=0),
                "image/width": tf.FixedLenFeature((), tf.int64, default_value=0),
                "image/channels": tf.FixedLenFeature((), tf.int64, default_value=0)}
    parsed_features = tf.parse_single_example(data, features)
    
    image = tf.image.decode_jpeg(parsed_features['image/encoded'])
    label = tf.cast(parsed_features["image/class/label"], tf.int64)
    width = tf.cast(parsed_features['image/width'], tf.int32)
    height = tf.cast(parsed_features['image/height'], tf.int32)
    channels = tf.cast(parsed_features["image/channels"], tf.int64)
    
    return image, label

def preprocess_dataset(images, labels) :
    images = tf.image.resize_images(images, (INPUT_SIZE, INPUT_SIZE))    
    images = tf.cast(images, tf.float32)
    images = tf.subtract(images, 128.0)
    images = tf.divide(images, 128.0)
    images = dataset_flip_left_and_right(images)
    images = dataset_rotate(images)
    images = dataset_zoom(images)
#     images = tf.divide(images, 255.0)
#     labels = tf.one_hot(labels, NUM_CLASSES) # Create a one hot array for your labels
    
    return images, labels

def dataset_rotate(images) :
    return tf.image.rot90(images, tf.random_uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

def dataset_flip_left_and_right(images) :
    return tf.image.random_flip_left_right(images)

def dataset_zoom(images) :
    scales = list(np.arange(0.7, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(images):
        # Create different crops for an image
        crops = tf.image.crop_and_resize([images], boxes=boxes, box_ind=np.zeros(len(scales)), crop_size=(INPUT_SIZE, INPUT_SIZE))
        # Return a random crop
        return crops[tf.random_uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]

    choice = tf.random_uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: images, lambda: random_crop(images))

In [5]:
def load_labels(label_path) :    
    labels = []
    with open(label_path, 'r') as f :
        for line in f.readlines() :
            labels.append(line.strip())

    return labels

def make_directory(dir_path) :
    if not os.path.isdir(dir_path):
        os.makedirs(dir_path)
        print("Directory is created in ", dir_path)

def show_images(image, label, width, height, channels) :
    label_names = load_labels(LABEL_PATH)
    
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    for idx in range(5) :
        if channels == 1 :
            image_data = tf.reshape(image[idx], [width[idx], height[idx]])
            axes[idx].imshow(image_data, cmap='gray')  
        else :
            image_data = image[idx]
            axes[idx].imshow(image_data)  

        axes[idx].set_title(label_names[label[idx]])

def get_dataset_steps(dataset) :
    counter = 0
    for _, _ in dataset :
        counter += 1
    return counter

def train_model(model, train_dataset, val_dataset, epoch=10000) :
    val_steps = get_dataset_steps(val_dataset)
    # print("Val dataset steps : %d" % val_steps)

    make_directory(CKPT_DIR)
        
    total_epoch = 0
    best_accuracy = 0

    for ep in range(epoch):
        for train_images, train_labels in train_dataset:
            train_loss, train_accuracy = model.train_on_batch(train_images, train_labels)
            total_epoch += 1

            if total_epoch % 100 == 0 :
                val_loss, val_accuracy = model.evaluate(val_dataset, steps=val_steps, verbose = 0)
                print('Epoch #%d    Loss: %.6f    Train Accuracy: %.6f    Val accuracy: %.6f' % (total_epoch, train_loss, train_accuracy, val_accuracy))

                if val_accuracy > best_accuracy :
                    model.save('%s/mobilenet_emotion_%.2f_%d.h5' % (CKPT_DIR, val_accuracy, total_epoch))
    #                 model.save_weights(CKPT_DIR + '/emotion_mobilenet_v1_{0:.2f}.h5'.format(val_accuracy))

                    print("Save model of accuracy %.6f" % val_accuracy)
                    best_accuracy = val_accuracy
    return model


In [6]:
train_dataset = create_dataset_from_tfrecord(TRAIN_OUTPUT_DIR_PATH, TRAIN_BATCH_SIZE)
val_dataset = create_dataset_from_tfrecord(VAL_OUTPUT_DIR_PATH, VAL_BATCH_SIZE)

if False : 
    model = keras.models.load_model('/work/nas/emotion/03_CheckPoint/smile_keras_test_190411/ckpt/emotion_mobilenet_v1_0.77.h5')
    model.compile(optimizer=tf.train.AdamOptimizer(0.001), 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
else :
    emonet = MobileNetV1(0.001)
    model = emonet.create_model()

model.summary()

model = train_model(model, train_dataset, val_dataset, BATCH_EPOCHS)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x (InputLayer)               (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 112, 112, 32)      320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 32)      0         
__________

Directory is created in  /work/nas/emotion/03_CheckPoint/smile_v2_224_mobilenet_190522/ckpt
Epoch #100    Loss: 0.637415    Train Accuracy: 0.687500    Val accuracy: 0.467333
Save model of accuracy 0.467333
Epoch #200    Loss: 0.458852    Train Accuracy: 0.781250    Val accuracy: 0.467333
Epoch #300    Loss: 0.715083    Train Accuracy: 0.687500    Val accuracy: 0.467333
Save model of accuracy 0.467333
Epoch #400    Loss: 0.618129    Train Accuracy: 0.750000    Val accuracy: 0.467333
Save model of accuracy 0.467333
Epoch #500    Loss: 0.555719    Train Accuracy: 0.718750    Val accuracy: 0.467333
Epoch #600    Loss: 0.564753    Train Accuracy: 0.718750    Val accuracy: 0.689990
Save model of accuracy 0.689990
Epoch #700    Loss: 0.427708    Train Accuracy: 0.781250    Val accuracy: 0.740692
Save model of accuracy 0.740692
Epoch #800    Loss: 0.648185    Train Accuracy: 0.687500    Val accuracy: 0.726930
Epoch #900    Loss: 0.679377    Train Accuracy: 0.593750    Val accuracy: 0.728958
E

Save model of accuracy 0.788643
Epoch #5600    Loss: 0.326227    Train Accuracy: 0.875000    Val accuracy: 0.774736
Epoch #5700    Loss: 0.260302    Train Accuracy: 0.843750    Val accuracy: 0.766333
Epoch #5800    Loss: 0.352231    Train Accuracy: 0.781250    Val accuracy: 0.712444
Epoch #5900    Loss: 0.559363    Train Accuracy: 0.718750    Val accuracy: 0.737361
Epoch #6000    Loss: 0.689658    Train Accuracy: 0.687500    Val accuracy: 0.771404
Epoch #6100    Loss: 0.178359    Train Accuracy: 1.000000    Val accuracy: 0.776184
Epoch #6200    Loss: 0.646314    Train Accuracy: 0.718750    Val accuracy: 0.716065
Epoch #6300    Loss: 0.721760    Train Accuracy: 0.750000    Val accuracy: 0.737940
Epoch #6400    Loss: 0.414267    Train Accuracy: 0.843750    Val accuracy: 0.719687
Epoch #6500    Loss: 0.165183    Train Accuracy: 0.968750    Val accuracy: 0.778647
Epoch #6600    Loss: 0.290707    Train Accuracy: 0.875000    Val accuracy: 0.661162
Epoch #6700    Loss: 0.249332    Train Accur

Epoch #12400    Loss: 0.087133    Train Accuracy: 0.968750    Val accuracy: 0.834565
Epoch #12500    Loss: 0.257868    Train Accuracy: 0.906250    Val accuracy: 0.832826
Epoch #12600    Loss: 0.207420    Train Accuracy: 0.906250    Val accuracy: 0.713458
Epoch #12700    Loss: 0.062970    Train Accuracy: 1.000000    Val accuracy: 0.810227
Epoch #12800    Loss: 0.159016    Train Accuracy: 0.968750    Val accuracy: 0.833551
Epoch #12900    Loss: 0.169565    Train Accuracy: 0.906250    Val accuracy: 0.794582
Epoch #13000    Loss: 0.320837    Train Accuracy: 0.875000    Val accuracy: 0.778212
Epoch #13100    Loss: 0.486204    Train Accuracy: 0.843750    Val accuracy: 0.765899
Epoch #13200    Loss: 0.229495    Train Accuracy: 0.906250    Val accuracy: 0.818340
Epoch #13300    Loss: 0.268853    Train Accuracy: 0.843750    Val accuracy: 0.768651
Epoch #13400    Loss: 0.219246    Train Accuracy: 0.906250    Val accuracy: 0.762132
Epoch #13500    Loss: 0.176623    Train Accuracy: 0.906250    Val

Epoch #20800    Loss: 0.143519    Train Accuracy: 0.937500    Val accuracy: 0.804578
Epoch #20900    Loss: 0.260051    Train Accuracy: 0.937500    Val accuracy: 0.838186
Epoch #21000    Loss: 0.282844    Train Accuracy: 0.875000    Val accuracy: 0.751268
Epoch #21100    Loss: 0.186459    Train Accuracy: 0.937500    Val accuracy: 0.817181
Epoch #21200    Loss: 0.194128    Train Accuracy: 0.937500    Val accuracy: 0.783572
Epoch #21300    Loss: 0.250073    Train Accuracy: 0.906250    Val accuracy: 0.814863
Epoch #21400    Loss: 0.077466    Train Accuracy: 1.000000    Val accuracy: 0.773866
Epoch #21500    Loss: 0.101442    Train Accuracy: 0.968750    Val accuracy: 0.790816
Epoch #21600    Loss: 0.354961    Train Accuracy: 0.843750    Val accuracy: 0.810372
Epoch #21700    Loss: 0.298818    Train Accuracy: 0.843750    Val accuracy: 0.812545
Epoch #21800    Loss: 0.164934    Train Accuracy: 0.937500    Val accuracy: 0.817616
Epoch #21900    Loss: 0.146279    Train Accuracy: 0.937500    Val

Epoch #30500    Loss: 0.058968    Train Accuracy: 1.000000    Val accuracy: 0.796755
Epoch #30600    Loss: 0.172803    Train Accuracy: 0.937500    Val accuracy: 0.829639
Epoch #30700    Loss: 0.186749    Train Accuracy: 0.906250    Val accuracy: 0.784586
Epoch #30800    Loss: 0.153600    Train Accuracy: 0.937500    Val accuracy: 0.828480
Epoch #30900    Loss: 0.259937    Train Accuracy: 0.843750    Val accuracy: 0.825004
Epoch #31000    Loss: 0.123882    Train Accuracy: 0.937500    Val accuracy: 0.788063
Epoch #31100    Loss: 0.115052    Train Accuracy: 0.937500    Val accuracy: 0.786470
Epoch #31200    Loss: 0.083645    Train Accuracy: 0.968750    Val accuracy: 0.791105
Epoch #31300    Loss: 0.324557    Train Accuracy: 0.937500    Val accuracy: 0.760104
Epoch #31400    Loss: 0.289749    Train Accuracy: 0.906250    Val accuracy: 0.786470
Epoch #31500    Loss: 0.056095    Train Accuracy: 1.000000    Val accuracy: 0.807765
Epoch #31600    Loss: 0.133428    Train Accuracy: 0.906250    Val

Epoch #40200    Loss: 0.063608    Train Accuracy: 0.968750    Val accuracy: 0.790381
Epoch #40300    Loss: 0.135311    Train Accuracy: 0.937500    Val accuracy: 0.767058
Epoch #40400    Loss: 0.118474    Train Accuracy: 0.937500    Val accuracy: 0.766044
Epoch #40500    Loss: 0.031357    Train Accuracy: 1.000000    Val accuracy: 0.814429
Epoch #40600    Loss: 0.143955    Train Accuracy: 0.968750    Val accuracy: 0.737650
Epoch #40700    Loss: 0.233802    Train Accuracy: 0.906250    Val accuracy: 0.791395
Epoch #40800    Loss: 0.121016    Train Accuracy: 0.937500    Val accuracy: 0.787484
Epoch #40900    Loss: 0.132946    Train Accuracy: 0.906250    Val accuracy: 0.813414
Epoch #41000    Loss: 0.121874    Train Accuracy: 0.937500    Val accuracy: 0.787049
Epoch #41100    Loss: 0.292923    Train Accuracy: 0.937500    Val accuracy: 0.747066
Epoch #41200    Loss: 0.084261    Train Accuracy: 1.000000    Val accuracy: 0.783862
Epoch #41300    Loss: 0.051459    Train Accuracy: 1.000000    Val

Epoch #49900    Loss: 0.104046    Train Accuracy: 0.968750    Val accuracy: 0.746487
Epoch #50000    Loss: 0.125168    Train Accuracy: 0.968750    Val accuracy: 0.737361
Epoch #50100    Loss: 0.104873    Train Accuracy: 0.937500    Val accuracy: 0.764740
Epoch #50200    Loss: 0.154962    Train Accuracy: 0.906250    Val accuracy: 0.790960
Epoch #50300    Loss: 0.051051    Train Accuracy: 1.000000    Val accuracy: 0.817181
Epoch #50400    Loss: 0.055973    Train Accuracy: 1.000000    Val accuracy: 0.801825
Epoch #50500    Loss: 0.394010    Train Accuracy: 0.875000    Val accuracy: 0.786470
Epoch #50600    Loss: 0.045241    Train Accuracy: 0.968750    Val accuracy: 0.805592
Epoch #50700    Loss: 0.160803    Train Accuracy: 0.906250    Val accuracy: 0.776619
Epoch #50800    Loss: 0.021586    Train Accuracy: 1.000000    Val accuracy: 0.800232
Epoch #50900    Loss: 0.030877    Train Accuracy: 1.000000    Val accuracy: 0.736202
Epoch #51000    Loss: 0.103863    Train Accuracy: 0.968750    Val

Epoch #59600    Loss: 0.116370    Train Accuracy: 0.968750    Val accuracy: 0.801391
Epoch #59700    Loss: 0.149622    Train Accuracy: 0.937500    Val accuracy: 0.758945
Epoch #59800    Loss: 0.205578    Train Accuracy: 0.875000    Val accuracy: 0.756917
Epoch #59900    Loss: 0.125928    Train Accuracy: 0.937500    Val accuracy: 0.756048
Epoch #60000    Loss: 0.074331    Train Accuracy: 0.968750    Val accuracy: 0.746052
Epoch #60100    Loss: 0.069510    Train Accuracy: 1.000000    Val accuracy: 0.779082
Epoch #60200    Loss: 0.156766    Train Accuracy: 0.968750    Val accuracy: 0.746342
Epoch #60300    Loss: 0.040933    Train Accuracy: 1.000000    Val accuracy: 0.753006
Epoch #60400    Loss: 0.053945    Train Accuracy: 0.968750    Val accuracy: 0.772563
Epoch #60500    Loss: 0.065610    Train Accuracy: 1.000000    Val accuracy: 0.737650
Epoch #60600    Loss: 0.102082    Train Accuracy: 1.000000    Val accuracy: 0.783427
Epoch #60700    Loss: 0.118715    Train Accuracy: 0.937500    Val

KeyboardInterrupt: 

In [ ]:
for val_images, val_labels in val_dataset :
    print(val_labels)
    prediction = model.predict(val_images)
    prediction = tf.argmax(prediction, 1)
    print(prediction)
    break